<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/selectivity_not_sorted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.manual_seed(1234)
np.random.seed(1234)

In [ ]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

In [ ]:
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid  = torch.nn.Sigmoid()

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid(x)
        pred = self.linear_2(x)

        return pred

In [ ]:
def get_activation(model):    
    def hook(module, input, output):
        model.layer_activations = output
    return hook

In [ ]:
no_epochs = 100
def selectivity_trainer(optimizer, model):

    hidden_layer_each_neuron = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
    hidden_layer_each_neuron = np.array(hidden_layer_each_neuron)

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    for epoch in range(no_epochs):
        total_train_loss = 0
        total_test_loss = 0

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

            # Print/Append activation of the hidden layer 
            # print(model.layer_activations.shape)
            # model.layer_activations

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # find selectivity at the final epoch 
            if epoch == no_epochs - 1: # last epoch 
                for activation, label in zip(model.layer_activations, labels):
                    # shape of activation and label: 256 and 1 
                    
                    # get the actual value of item. This is because label is now Tensor 
                    label = label.item()

                    # this is not part of gradient calculcation 
                    with torch.no_grad():
                        activation = activation.numpy()

                    # for each image/label, append activation value of neuron 
                    for i in range(256):    # number of neurons in hidden layer 
                        hidden_layer_each_neuron[i][label].append(activation[i])


        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

        # if total_test_loss < best_test_loss:
        #     best_test_loss = total_test_loss
        #     print("Saving the model state dictionary for Epoch: {} with Test loss: {:.8f}".format(epoch + 1, total_test_loss))
        #     torch.save(model.state_dict(), "model.dth")

    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(256)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(256)]

    # selectivity_list contains all of the selectivity of each neuron 
    selectivity_list = list()

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)


    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity_list
        selectivity_list.append(selectivity)

    return test_acc, selectivity_list

# AdaDelta

In [ ]:
model_Adadelta = Model()
print("model_Adadelta:", model_Adadelta)
model_Adadelta.to(device)
model_Adadelta.sigmoid.register_forward_hook(get_activation(model_Adadelta))
optimizer_Adadelta = torch.optim.Adadelta(model_Adadelta.parameters(), lr=1.0)
Adadelta_test_acc, Adadelta_selectivity_list = selectivity_trainer(optimizer=optimizer_Adadelta, model=model_Adadelta)

f = open("not_sorted_selectivity_Adadelta.txt", "a")
f.write(str(0)+'\n'+str(Adadelta_test_acc)+'\n'+str(np.average(Adadelta_selectivity_list))+'\n'+str(np.std(Adadelta_selectivity_list))+'\n\n')
f.close()

!cp not_sorted_selectivity_Adadelta.txt /content/drive/MyDrive

# AdaGrad

In [ ]:
model_Adagrad = Model()
print("model_Adagrad:", model_Adagrad)
model_Adagrad.to(device)
model_Adagrad.sigmoid.register_forward_hook(get_activation(model_Adagrad))
optimizer_Adagrad = torch.optim.Adagrad(model_Adagrad.parameters(), lr=0.1)
Adagrad_test_acc, Adagrad_selectivity_list = selectivity_trainer(optimizer=optimizer_Adagrad, model=model_Adagrad)

f = open("not_sorted_selectivity_Adagrad.txt", "w")
f.write(str(0)+'\n'+str(Adagrad_test_acc)+'\n'+str(np.average(Adagrad_selectivity_list))+'\n'+str(np.std(Adagrad_selectivity_list))+'\n\n')
f.close()

!cp not_sorted_selectivity_Adagrad.txt /content/drive/MyDrive

model_Adagrad: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.25258046, Test Loss: 0.12609529, Test Accuracy: 0.96400000

Epoch: 2/100, Train Loss: 0.10309967, Test Loss: 0.10109970, Test Accuracy: 0.96870000

Epoch: 3/100, Train Loss: 0.07298480, Test Loss: 0.09385988, Test Accuracy: 0.96920000

Epoch: 4/100, Train Loss: 0.05632330, Test Loss: 0.07996433, Test Accuracy: 0.97410000

Epoch: 5/100, Train Loss: 0.04489036, Test Loss: 0.07642399, Test Accuracy: 0.97640000

Epoch: 6/100, Train Loss: 0.03629026, Test Loss: 0.07400378, Test Accuracy: 0.97730000

Epoch: 7/100, Train Loss: 0.03001808, Test Loss: 0.07167674, Test Accuracy: 0.97710000

Epoch: 8/100, Train Loss: 0.02490960, Test Loss: 0.07024349, Test Accuracy: 0.97810000

Epoch: 9/100, Train Loss: 0.02092683, Test Loss: 0.06943788, Test Accuracy: 0.97850000

Epoch: 10/100, Train Loss: 

# SGD


In [ ]:
model_SGD = Model()
print("model_SGD:", model_SGD)
model_SGD.to(device)
model_SGD.sigmoid.register_forward_hook(get_activation(model_SGD))
optimizer_SGD = torch.optim.SGD(model_SGD.parameters(), lr=0.1)
SGD_test_acc, SGD_selectivity_list = selectivity_trainer(optimizer=optimizer_SGD, model=model_SGD)

f = open("not_sorted_selectivity_SGD.txt", "w")
f.write(str(0)+'\n'+str(SGD_test_acc)+'\n'+str(np.average(SGD_selectivity_list))+'\n'+str(np.std(SGD_selectivity_list))+'\n\n')
f.close()

!cp not_sorted_selectivity_SGD.txt /content/drive/MyDrive

model_SGD: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.77723409, Test Loss: 0.37313266, Test Accuracy: 0.89520000

Epoch: 2/100, Train Loss: 0.34727685, Test Loss: 0.30445907, Test Accuracy: 0.91280000

Epoch: 3/100, Train Loss: 0.30735245, Test Loss: 0.28385138, Test Accuracy: 0.92000000

Epoch: 4/100, Train Loss: 0.28485614, Test Loss: 0.26544578, Test Accuracy: 0.92490000

Epoch: 5/100, Train Loss: 0.26737319, Test Loss: 0.25191398, Test Accuracy: 0.92810000

Epoch: 6/100, Train Loss: 0.25029423, Test Loss: 0.24011335, Test Accuracy: 0.93030000

Epoch: 7/100, Train Loss: 0.23442947, Test Loss: 0.22011212, Test Accuracy: 0.93650000

Epoch: 8/100, Train Loss: 0.21924428, Test Loss: 0.21427857, Test Accuracy: 0.93850000

Epoch: 9/100, Train Loss: 0.20613533, Test Loss: 0.20255191, Test Accuracy: 0.94080000

Epoch: 10/100, Train Loss: 0.19

# Adam

In [ ]:
model_Adam = Model()
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, Adam_selectivity_list = selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam)

f = open("not_sorted_selectivity_Adam.txt", "w")
f.write(str(0)+'\n'+str(Adam_test_acc)+'\n'+str(np.average(Adam_selectivity_list))+'\n'+str(np.std(Adam_selectivity_list))+'\n\n')
f.close()

!cp not_sorted_selectivity_Adam.txt /content/drive/MyDrive

model_Adam: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.42718115, Test Loss: 0.22443770, Test Accuracy: 0.93520000

Epoch: 2/100, Train Loss: 0.19834835, Test Loss: 0.16388322, Test Accuracy: 0.95140000

Epoch: 3/100, Train Loss: 0.14312717, Test Loss: 0.13139713, Test Accuracy: 0.96030000

Epoch: 4/100, Train Loss: 0.10981040, Test Loss: 0.10850899, Test Accuracy: 0.96910000

Epoch: 5/100, Train Loss: 0.08679321, Test Loss: 0.09350853, Test Accuracy: 0.97100000

Epoch: 6/100, Train Loss: 0.06963237, Test Loss: 0.08395875, Test Accuracy: 0.97470000

Epoch: 7/100, Train Loss: 0.05673326, Test Loss: 0.07634868, Test Accuracy: 0.97660000

Epoch: 8/100, Train Loss: 0.04688189, Test Loss: 0.07065204, Test Accuracy: 0.97760000

Epoch: 9/100, Train Loss: 0.03782095, Test Loss: 0.07152031, Test Accuracy: 0.97810000

Epoch: 10/100, Train Loss: 0.0